In [2]:
import gym
import numpy as np
import torch
import matplotlib.pyplot as plt
import time

In [3]:
from gym.wrappers import Monitor

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch import pow, add, mul, div, sqrt

In [5]:
import math
import copy
import numpy as np

In [6]:
import flagreader
import datareader

flags = flagreader.read_flag()
train_loader, test_loader = datareader.read_data(x_range=flags.x_range,
                                                     y_range=[i for i in range(20 , 320 )],
                                                     geoboundary=flags.geoboundary,
                                                     batch_size=flags.batch_size,
                                                     normalize_input=flags.normalize_input,
                                                     data_dir=flags.data_dir,
                                                     test_ratio=0.999)

Importing data files...
['ToyModelSim_e2_with_params_16.csv', 'ToyModelSim_e2_with_params_3.csv', 'ToyModelSim_e2_with_params_10.csv', 'ToyModelSim_e2_with_params_12.csv', 'ToyModelSim_e2_with_params_15.csv', 'ToyModelSim_e2_with_params_11.csv', 'ToyModelSim_e2_with_params_4.csv', 'ToyModelSim_e2_with_params_23.csv', 'ToyModelSim_e2_with_params_7.csv', 'ToyModelSim_e2_with_params_25.csv', 'ToyModelSim_e2_with_params_9.csv', 'ToyModelSim_e2_with_params_8.csv', 'ToyModelSim_e2_with_params_6.csv', 'ToyModelSim_e2_with_params_13.csv', 'ToyModelSim_e2_with_params_2.csv', 'ToyModelSim_e2_with_params_24.csv', 'ToyModelSim_e2_with_params_22.csv', 'ToyModelSim_e2_with_params_5.csv', 'ToyModelSim_e2_with_params_1.csv', 'ToyModelSim_e2_with_params_18.csv', 'ToyModelSim_e2_with_params_17.csv', 'ToyModelSim_e2_with_params_19.csv', 'ToyModelSim_e2_with_params_21.csv', 'ToyModelSim_e2_with_params_20.csv', 'ToyModelSim_e2_with_params_14.csv']
Splitting data into training and test sets with a ratio of:

In [8]:
class LorentzNN(nn.Module):
    def __init__(self, flags):
        super(LorentzNN, self).__init__()
        self.flags = flags

        w_numpy = np.arange(flags.freq_low, flags.freq_high,
                            (flags.freq_high - flags.freq_low) / self.flags.num_spec_points)

        cuda = True if torch.cuda.is_available() else False
        if cuda:
            self.w = torch.tensor(w_numpy).cuda()
        else:
            self.w = torch.tensor(w_numpy)

        self.linears = nn.ModuleList([])
        for ind, fc_num in enumerate(flags.linear[0:-1]):
            self.linears.append(nn.Linear(fc_num, flags.linear[ind + 1], bias=True))

        layer_size = flags.linear[-1]
        self.lin_w0 = nn.Linear(layer_size, self.flags.num_lorentz_osc, bias=False)
        self.lin_wp = nn.Linear(layer_size, self.flags.num_lorentz_osc, bias=False)
        self.lin_g = nn.Linear(layer_size, self.flags.num_lorentz_osc, bias=False)

    def forward(self, G):
        out = G
        for ind, fc in enumerate(self.linears):
            if ind <= len(self.linears):
                out = F.relu(fc(out))

        w0 = F.relu(self.lin_w0(out))
        wp = F.relu(self.lin_wp(out))
        g = F.relu(self.lin_g(out))

        w0 = w0.unsqueeze(2) * 1
        wp = wp.unsqueeze(2) * 1
        g = g.unsqueeze(2) * 0.1

        w0 = w0.expand(out.size(0), self.flags.num_lorentz_osc, self.flags.num_spec_points)
        wp = wp.expand_as(w0)
        g = g.expand_as(w0)
        w_expand = self.w.expand_as(g)

        e2 = div(mul(pow(wp, 2), mul(w_expand, g)),
                 add(pow(add(pow(w0, 2), -pow(w_expand, 2)), 2), mul(pow(w_expand, 2), pow(g, 2))))

        e2 = torch.sum(e2, 1)
        return e2.float()

In [8]:
class CartPoleAI(nn.Module):
        def __init__(self):
            super().__init__()
            self.fc = nn.Sequential(
                        nn.Linear(4,128, bias=True),
                        nn.ReLU(),
                        nn.Linear(128,2, bias=True),
                        nn.Softmax(dim=1)
                        )


        def forward(self, inputs):
            x = self.fc(inputs)
            return x


In [9]:
def init_weights(m):

        # nn.Conv2d weights are of shape [16, 1, 3, 3] i.e. # number of filters, 1, stride, stride
        # nn.Conv2d bias is of shape [16] i.e. # number of filters

        # nn.Linear weights are of shape [32, 24336] i.e. # number of input features, number of output features
        # nn.Linear bias is of shape [32] i.e. # number of output features

        if ((type(m) == nn.Linear) | (type(m) == nn.Conv2d)):
            torch.nn.init.xavier_uniform(m.weight)
            if m.bias:
                m.bias.data.fill_(0.00)

In [10]:
def return_random_agents(num_agents):

    agents = []
    for _ in range(num_agents):

        agent = LorentzNN(flags)

        for param in agent.parameters():
            param.requires_grad = False

        init_weights(agent)
        agents.append(agent)


    return agents

In [11]:
def run_agents(agents):

    reward_agents = []
    for agent_num, agent in enumerate(agents):
        cuda = True if torch.cuda.is_available() else False
        if cuda:
            agent.cuda()
        agent.eval()

        eval_loss = []
        with torch.no_grad():
            for ind, (geometry, spectra) in enumerate(test_loader):
                if cuda:
                    geometry = geometry.cuda()
                    spectra = spectra.cuda()
                logit = agent(geometry)
                loss = nn.functional.mse_loss(logit, spectra)
                eval_loss.append(np.copy(loss.cpu().data.numpy()))
        eval_avg_loss = np.mean(eval_loss)
        reward = 1/eval_avg_loss
        reward_agents.append(reward)
        # print(agent_num)

    return reward_agents

In [13]:
def return_average_score(agent, runs):
    score = 0.
    for i in range(runs):
        score += run_agents([agent])[0]
    return score/runs

In [14]:
def run_agents_n_times(agents, runs):
    avg_score = []
    for agent in agents:
        avg_score.append(return_average_score(agent,runs))
    return avg_score

In [15]:
def mutate(agent):

    child_agent = copy.deepcopy(agent)

    mutation_power = 0.02 #hyper-parameter, set from https://arxiv.org/pdf/1712.06567.pdf

    for param in child_agent.parameters():

        if(len(param.shape)==4): #weights of Conv2D

            for i0 in range(param.shape[0]):
                for i1 in range(param.shape[1]):
                    for i2 in range(param.shape[2]):
                        for i3 in range(param.shape[3]):

                            param[i0][i1][i2][i3]+= mutation_power * np.random.randn()



        elif(len(param.shape)==2): #weights of linear layer
            for i0 in range(param.shape[0]):
                for i1 in range(param.shape[1]):

                    param[i0][i1]+= mutation_power * np.random.randn()


        elif(len(param.shape)==1): #biases of linear layer or conv layer
            for i0 in range(param.shape[0]):

                param[i0]+=mutation_power * np.random.randn()

    return child_agent

In [16]:
def return_children(agents, sorted_parent_indexes, elite_index):

    children_agents = []

    #first take selected parents from sorted_parent_indexes and generate N-1 children
    for i in range(len(agents)-1):

        selected_agent_index = sorted_parent_indexes[np.random.randint(len(sorted_parent_indexes))]
        children_agents.append(mutate(agents[selected_agent_index]))

    #now add one elite
    elite_child = add_elite(agents, sorted_parent_indexes, elite_index)
    children_agents.append(elite_child)
    elite_index=len(children_agents)-1 #it is the last one

    return children_agents, elite_index


In [17]:
def add_elite(agents, sorted_parent_indexes, elite_index=None, only_consider_top_n=10):

    candidate_elite_index = sorted_parent_indexes[:only_consider_top_n]

    if(elite_index is not None):
        candidate_elite_index = np.append(candidate_elite_index,[elite_index])

    top_score = None
    top_elite_index = None

    for i in candidate_elite_index:
        score = return_average_score(agents[i],runs=5)
        print("Score for elite i ", i, " is ", score)

        if(top_score is None):
            top_score = score
            top_elite_index = i
        elif(score > top_score):
            top_score = score
            top_elite_index = i

    print("Elite selected with index ",top_elite_index, " and score", top_score)

    child_agent = copy.deepcopy(agents[top_elite_index])
    return child_agent


In [18]:
def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    return np.exp(x) / np.sum(np.exp(x), axis=0)



In [22]:
# game_actions = 2 #2 actions possible: left or right

#disable gradients as we will not use them
torch.set_grad_enabled(False)

# initialize N number of agents
num_agents = 500
agents = return_random_agents(num_agents)

# How many top agents to consider as parents
top_limit = 20

# run evolution until X generations
generations = 1000

elite_index = None

for generation in range(generations):

    # return rewards of agents
    # rewards = run_agents_n_times(agents, 1) #return average of 3 runs
    rewards = run_agents(agents)

    # sort by rewards
    sorted_parent_indexes = np.argsort(rewards)[::-1][:top_limit] #reverses and gives top values (argsort sorts by ascending by default) https://stackoverflow.com/questions/16486252/is-it-possible-to-use-argsort-in-descending-order
    print("")
    print("")

    top_rewards = []
    for best_parent in sorted_parent_indexes:
        top_rewards.append(rewards[best_parent])

    print("Generation ", generation, " | Mean rewards: ", np.mean(rewards), " | Mean of top 5: ",np.mean(top_rewards[:5]))
    #print(rewards)
    print("Top ",top_limit," scores", sorted_parent_indexes)
    print("Rewards for top: ",top_rewards)

    # setup an empty list for containing children agents
    children_agents, elite_index = return_children(agents, sorted_parent_indexes, elite_index)

    # kill all agents, and replace them with their children
    agents = children_agents




Generation  0  | Mean rewards:  0.01757679456561695  | Mean of top 5:  0.01757685031011232
Top  5  scores [5 2 4 1 3]
Rewards for top:  [0.017576966278272644, 0.017576925029128478, 0.017576815425200374, 0.017576777712336893, 0.017576767105623198]
Score for elite i  5  is  0.017576966278272644
Score for elite i  2  is  0.017576925029128478
Score for elite i  4  is  0.017576815425200374
Score for elite i  1  is  0.017576777712336893
Score for elite i  3  is  0.017576767105623198
Elite selected with index  5  and score 0.017576966278272644


Generation  1  | Mean rewards:  0.017576852667371506  | Mean of top 5:  0.01757694105755998
Top  5  scores [7 9 4 5 3]
Rewards for top:  [0.017577046420020496, 0.017576966278272644, 0.01757692856476182, 0.017576888494333896, 0.017576875530411037]
Score for elite i  7  is  0.017577046420020496
Score for elite i  9  is  0.017576966278272644
Score for elite i  4  is  0.01757692856476182
Score for elite i  5  is  0.017576888494333896
Score for elite i  

KeyboardInterrupt: 

In [19]:
def play_agent(agent):
    try: #try and exception block because, render hangs if an erorr occurs, we must do env.close to continue working
        env = gym.make("CartPole-v0")

        env_record = Monitor(env, './video', force=True)
        observation = env_record.reset()
        last_observation = observation
        r=0
        for _ in range(250):
            env_record.render()
            inp = torch.tensor(observation).type('torch.FloatTensor').view(1,-1)
            output_probabilities = agent(inp).detach().numpy()[0]
            action = np.random.choice(range(game_actions), 1, p=output_probabilities).item()
            new_observation, reward, done, info = env_record.step(action)
            r=r+reward
            observation = new_observation

            if(done):
                break

        env_record.close()
        print("Rewards: ",r)

    except Exception as e:
        env_record.close()
        print(e.__doc__)
        print(e.message)


In [31]:
play_agent(agents[30])



Rewards:  86.0
